# Utilizing the MtG Gatherer API I will grab all card data

In [1]:
%pip install mtgsdk

from mtgsdk import Card
from mtgsdk import Set
from mtgsdk import Type
from mtgsdk import Supertype
from mtgsdk import Subtype
from mtgsdk import Changelog

Note: you may need to restart the kernel to use updated packages.


## Card Properties Per Class

    name
    multiverse_id
    layout
    names
    mana_cost
    cmc
    colors
    color_identity
    type
    supertypes
    subtypes
    rarity
    text
    flavor
    artist
    number
    power
    toughness
    loyalty
    variations
    watermark
    border
    timeshifted
    hand
    life
    reserved
    release_date
    starter
    rulings
    foreign_names
    printings
    original_text
    original_type
    legalities
    source
    image_url
    set
    set_name
    id  

# Card Set Info

    code
    name
    gatherer_code
    old_code
    magic_cards_info_code
    release_date
    border
    type
    block
    online_only
    booster
    mkm_id
    mkm_name

In [6]:
card = Card.find(386615)

In [3]:
card.image_url

'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=386616&type=card'

# The Gatherer API is not providing high quality images

I will use a bulk download from Scryfall.com found here https://scryfall.com/docs/api/bulk-data

The dataset I am using is the "Default Cards" which is a JSON file containing every card object on Scryfall in English or the printed language if the card is only available in one language.

With this, we can grab each image and save it for use in the model.


In [1]:
import json

# Load the JSON file
with open('./Datasets/default-cards-20240817211226.json', 'r') as file:
    cards_data = json.load(file)


## Let us look at the data a bit to see what we are working with

In [7]:
print('Length of Dataset:' + str(len(cards_data)))
print('Number of features per card:' + str(len(cards_data[0])))
print('Features:' + str(cards_data[0].keys()))

Length of Dataset:99238
Number of features per card:59
Features:dict_keys(['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id', 'arena_id', 'tcgplayer_id', 'name', 'lang', 'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris', 'mana_cost', 'cmc', 'type_line', 'oracle_text', 'colors', 'color_identity', 'keywords', 'produced_mana', 'legalities', 'games', 'reserved', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'collector_number', 'digital', 'rarity', 'card_back_id', 'artist', 'artist_ids', 'illustration_id', 'border_color', 'frame', 'full_art', 'textless', 'booster', 'story_spotlight', 'prices', 'related_uris', 'purchase_uris'])


## I am curious to see if any cards do not have an image associated with them